# Práctica 2
## Parte 2. Recuperación de Información
<br>

__Alumnos:__
* __Frederick Ernesto Borges Noronha__
* __Victor Manuel Cavero Gracia__

Aquí vamos a utilizar el conjunto de datos de 20 Newsgroups que se encuentra disponible en Scikit-learn y que hemos usado en el notebook de ejemplo.

El conjunto consiste en textos de un foro sobre diferentes temas, desde hardware hasta religión. Algunos temas están muy relacionados, p.ej. "IBM PC hardware" y "Mac hardware", mientras que otros son más diversos, p.ej. "religion" o "hockey").

El objetivo de esta parte es poner en poner en práctica los conceptos de recuperación de información para realizar un buscador de mensajes en un foro.

### Apartado A

El conjunto está dividido de forma predeterminada en entrenamiento y prueba en porcentajes de 60 y 40, respectivamente (como se puede ver en el notebook de prueba). Usaremos únicamente la parte de entrenamiento como los mensajes a recuperar por nuestro buscador.

Vamos a utilizar una representación de la bolsa de palabras de **countVectorizer** con las siguientes opciones:
- La bolsa de palabras tendrá en cuenta la frecuencia de las palabras en cada mensaje (**binary=False**)
- Usa el diccionario que se encuentra en la siguiente URL y que ya usamos en el notebook de prueba. https://github.com/dwyl/english-words/blob/master/words.txt
- Usa la lista de palabras vacías (parámetro **stop_words**) que proporciona sklearn para el inglés
- Usando un rango de n-gramas de (1,1) (parámetro **ngram_range**).

Para calcular la similitud entre dos mensajes usaremos la similitud del coseno (**sklearn.metrics.pairwise.cosine_similarity**) que es capaz de medir la similitud entre los elementos (es decir, entre las filas) de dos matrices de vectores de términos pudiendo ser estas matrices densas o dispersas.

Toma 3 mensaje del conjunto de prueba para cada clase (es decir, para cada tema). Vas a usar cada uno de dichos mensajes como consulta para recuperar los mensajes del conjunto de entrenamiento que más se parezcan a la consulta. Para ello sigue los siguientes pasos:

1. Usa la distancia del coseno entre el mensaje de consulta y los mensajes de entrenamiento.

2. Ordena los resultados de mayor a menor relevancia con la consulta.

3. Calcula la precisión de la lista de resultados con nivel de exhaustividad 3 y 10. 

    - La precisión a un nivel de exhaustividad X es el número de resultados que son relevantes (es decir, de la clase buscada) de entre los X primeros recuperados.

4. Calcula los valores de precisión media (para cada nivel de exhaustividad) para cada clase del conjunto de datos.

Se valorará el uso de funciones y la claridad del código, así como sus comentarios. Contesta a lo siguiente:

- ¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?

- Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.

    - ¿Con qué clases se ha confundido más dicha consulta? 

    - ¿A qué crees que se deben los malos resultados?

Debes usar la parte de entrenamiento para construir la bolsa de palabras con frecuencia y bolsa de palabras con TF/IDF.

### Apartado B

Repite la secuencia de pasos descritos en el apartado a) pero ahora usa TF-IDF para ponderar el peso de los términos de la bolsa de palabras.  Para usar TF-IDF primero debes transformar los textos  usando  **countVectorizer** con **binary=False**  para  obtener  la  frecuencia  de  palabras  (exactamente igual que en el apartado anterior), y a continuación usar **TfidfTransformer** para modular dicha frecuencia según lo popular que sea cada término en el conjunto de mensajes de entrenamiento.

A continuación contesta a lo siguiente.

- ¿Han cambiado los valores de precisión media para las clases del conjunto de datos?¿Qué clases han mejorado? ¿Cuáles han empeorado?

- Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.

### Imports:
A continuación se encuentran todos los imports de las librerias de las que haremos uso en esta práctica.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import operator

## Definición de Funciones

In [2]:
documentos_guardados = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
consultas_a_generar = fetch_20newsgroups(subset='test')

print("Saved documents:", len(documentos_guardados.data))
print("# of consults:", len(consultas_a_generar.data))

Saved documents: 11314
# of consults: 7532


In [3]:
# Crear el diccionario de palabras
with open('Datos/words.txt') as f:
    dictionary = f.read().splitlines()

print("Dictionary Length: ", len(dictionary), " words")

Dictionary Length:  466551  words


In [4]:
def obtenerNombresDeClase(target,mensajes):
    '''
    Obtiene una tupla con el ID de la clase y 
    el nombre de la clase a la que pertenece.
    '''
    return (target, mensajes.target_names[target])

In [5]:
def todosLosTemasTienenNElementos(temas, n):
    '''
    Comprueba que todos los temas tenga N cantidad de elementos.
    '''
    for key, value in temas.items():
        if (len(value) != n):
            return False
    return True

In [6]:
def buscarNMensajesPorTema(temas_consultas, mensajes_consultas, n):
    '''
    Genera un diccionario de la forma 
    {"Tema": ("ID_Mensaje", "Mensaje")}
    '''
    temas = {x: [] for x in consultas_a_generar.target_names}
    i = 0
    while (not todosLosTemasTienenNElementos(temas, n)):
        if (len(temas[temas_consultas[i]]) < n):
            value = (i, mensajes_consultas[i])
            temas[temas_consultas[i]].append(value)
        i += 1
    return temas

In [7]:
def recuperarNResultados(consulta,documentos, n):
    '''
    Obtenemos los n similitudes del coseno entre la consulta y los documentos, además ordenamos los valores
    '''
    x = cosine_similarity(consulta,documentos)[0]
    y = [index for index in range(len(x))]
    values = list(zip(y,x))
    values.sort(reverse=True, key = operator.itemgetter(1))
    return values[0:n]

In [8]:
def calcularTargets(consulta, documentos, documentos_guardados, n):
    '''
    Crea la lista de los targets, ademas de la similitud del coseno y las ids
    '''
    resultado = recuperarNResultados(consulta,documentos, n)
    ids, cosineSimilarity = zip(*resultado)
    targets = []
    for id in ids:
        targets.append(documentos_guardados.target[id])
    
    return targets, cosineSimilarity, ids

In [9]:
def precision(numRelevantDocs, numDocsObtained):
    '''
    Clases acertadas entre el numero de clases totales
    '''
    return numRelevantDocs/numDocsObtained

In [10]:
def getNumRelevantDocs(claseBuscada, clasesObtenidas):
    '''
    Cálculo del número de clases acertadas
    '''
    count = 0
    for clase in clasesObtenidas:
        if (clase == claseBuscada):
            count += 1
    return count

In [11]:
def calcularPrecision(consulta, targetConsulta, documentos, documentos_guardados, n):
    '''
    Calcula la precisión de los resultados obtenidos
    '''
    targets, cosineSimilarity, ids = calcularTargets(consulta, documentos, documentos_guardados, n)
    ret = precision(getNumRelevantDocs(targetConsulta, targets),n)
    return ret, ids

In [12]:
def generarResultados(nMensajes, topics, documentos, documentos_guardados, ids, n):
    '''
    Creo los resultados finales, es decir, dos diccionarios uno para las ids de los mensajes y otro
    los calculos de la similitud del coseno
    '''
    resultDict = dict()
    idsDict = dict()
    for key, values in nMensajes.items():
        resultDict[topics[values[0][0]]] = np.array([])
        idsDict[topics[values[0][0]]] = []
        for id, value in values:
            precision, ids_mensajes = calcularPrecision(value, ids[id], documentos, documentos_guardados, n)
            idsDict[topics[id]].append(ids_mensajes)
            resultDict[topics[id]] = np.append(resultDict[topics[id]], precision)
            
    return resultDict, idsDict

In [13]:
def imprimirResultado(resultados):
    '''
    Muestra los resultados de la similitud del coseno para todas las clases,
    además realiza su media.
    '''
    print("Resultados Precisión\n")
    for key, value in resultados.items():
        print("{}: \n\t {} - Precisión = {}".format(key,value,value.mean()) )

In [14]:
def ejecutarBuscador(consultas_a_generar, consultas, documentos_guardados, documentos,n=3, numMensajes=3):
    '''
    Ejecuta el buscador utilizando el resto de funciones descritas anteriormente, para finalizar devolvemos
    la lista con los resultados finales, un diccionario con las clases y las ids que se han escogido
    y otro diccionario que contiene las clases con la id de los mensajes que se le han asignado y el contenido
    de dichos mensajes. (Estos dos últimos resultados los utilizamos principalmente para mostrar los mensajes y 
    poder comparar la efectividad de los clasificadores)
    '''
    id_and_topics = [obtenerNombresDeClase(x,consultas_a_generar) for x in consultas_a_generar.target]
    ids, topics = zip(*id_and_topics)
    nMensajes = buscarNMensajesPorTema(topics, consultas, numMensajes)
    resultados, idsDict = generarResultados(nMensajes, topics, documentos, documentos_guardados, ids, n)
    imprimirResultado(resultados)
    return resultados, idsDict, nMensajes

### Códigos necesarios para el análisis de los resultados

In [15]:
def obtenerClaseConID(id, mensajes):
    return mensajes.target_names[mensajes.target[id]]

In [16]:
def comparar(resultado, resultadoTFIDF):
    '''
    Cálculo de la tasa de mejora de las distintas clases. La precisión de la clase ha mejorado si obtiene un valor positivo
    y ha empeorado en caso contrario (Diferencia entre la media de precisiones). 
    '''
    columnas = ["Clase", "Mejora"]
    diccionario = {"Clase": [], "Mejora": []}
    for key, value in resultado.items():
        diccionario["Clase"].append(key)
        diccionario["Mejora"].append(resultadoTFIDF[key].mean() - value.mean())
    df = pd.DataFrame(data=diccionario)
    return df

In [17]:
def imprimirConsulta(consultas_a_generar,ids):
    '''
    Imprime el mensaje utilizado para la consulta
    '''
    print("Clase Consultada: {}\n\n".format(obtenerClaseConID(ids,consultas_a_generar)))
    print("Mensaje Consultado: \n\n{}\n\n".format(consultas_a_generar.data[ids])) 

In [18]:
def imprimirResultados(documentos_guardados,ids):
    '''
    Imprime el mensaje obtenido del conjunto de entrenamiento gracias a la consulta
    '''
    print("Clase Obtenida: {}\n\n".format(obtenerClaseConID(ids,documentos_guardados)))
    print("Mensaje Obtenido: \n\n{}\n\n".format(documentos_guardados.data[ids])) 

In [19]:
def imprimirMensajes(consultas_a_generar, documentos_guardados,clase, 
                     id_resultados, precision_resultados, nMensajes, comparador=min):
    '''
    Imprime inicialmente el mensaje de la consulta y luego los mensajes obtenidos para esta.
    Utiliza como criterio de elección la variable 'comparador' para elegir la precisión 
    máxima o mínima según nos interese.
    '''
    ids = list(precision_resultados[clase]).index(comparador(precision_resultados[clase]))
    lista = id_resultados[clase][ids]
    id_consulta = nMensajes[clase][ids][0]
    imprimirConsulta(consultas_a_generar,id_consulta)
    for i in lista:
        imprimirResultados(documentos_guardados,i)

## Apartado A

In [20]:
# Creamos el CountVectorizer

vectorizer = CountVectorizer(vocabulary=dictionary, stop_words='english', ngram_range=(1,1), binary=False)
documentos=vectorizer.fit_transform(documentos_guardados.data)

In [21]:
# Tomamos los textos del conjunto de test y los transformamos en una matriz
# de palabras. Al usar "transform" toma como referencia únicamente las palabras
# encontradas en el conjunto de entrenamiento
consultas=vectorizer.transform(consultas_a_generar.data)

In [22]:
resultadoEx3, idsEx3, nMensajesEx3 = ejecutarBuscador(consultas_a_generar,consultas, documentos_guardados,documentos,3)

Resultados Precisión

alt.atheism: 
	 [1. 0. 0.] - Precisión = 0.3333333333333333
comp.graphics: 
	 [0.66666667 0.         0.        ] - Precisión = 0.2222222222222222
comp.os.ms-windows.misc: 
	 [0.66666667 0.         0.        ] - Precisión = 0.2222222222222222
comp.sys.ibm.pc.hardware: 
	 [1.         0.         0.66666667] - Precisión = 0.5555555555555555
comp.sys.mac.hardware: 
	 [0. 0. 0.] - Precisión = 0.0
comp.windows.x: 
	 [0.         0.33333333 0.66666667] - Precisión = 0.3333333333333333
misc.forsale: 
	 [0.66666667 0.66666667 1.        ] - Precisión = 0.7777777777777777
rec.autos: 
	 [0. 0. 1.] - Precisión = 0.3333333333333333
rec.motorcycles: 
	 [1.         1.         0.66666667] - Precisión = 0.8888888888888888
rec.sport.baseball: 
	 [1.         0.66666667 0.66666667] - Precisión = 0.7777777777777777
rec.sport.hockey: 
	 [0.33333333 0.66666667 1.        ] - Precisión = 0.6666666666666666
sci.crypt: 
	 [1.         0.33333333 0.33333333] - Precisión = 0.5555555555555555
sci.

In [23]:
resultadoEx10, idsEx10, nMensajesEx10 = ejecutarBuscador(consultas_a_generar,consultas, documentos_guardados,documentos,10)

Resultados Precisión

alt.atheism: 
	 [0.7 0.  0. ] - Precisión = 0.2333333333333333
comp.graphics: 
	 [0.6 0.  0. ] - Precisión = 0.19999999999999998
comp.os.ms-windows.misc: 
	 [0.8 0.1 0. ] - Precisión = 0.3
comp.sys.ibm.pc.hardware: 
	 [1.  0.1 0.4] - Precisión = 0.5
comp.sys.mac.hardware: 
	 [0.1 0.1 0. ] - Precisión = 0.06666666666666667
comp.windows.x: 
	 [0.1 0.4 0.4] - Precisión = 0.3
misc.forsale: 
	 [0.5 0.6 0.6] - Precisión = 0.5666666666666668
rec.autos: 
	 [0.  0.  0.9] - Precisión = 0.3
rec.motorcycles: 
	 [0.9 1.  0.3] - Precisión = 0.7333333333333333
rec.sport.baseball: 
	 [1.  0.8 0.5] - Precisión = 0.7666666666666666
rec.sport.hockey: 
	 [0.2 0.6 0.9] - Precisión = 0.5666666666666668
sci.crypt: 
	 [0.8 0.2 0.1] - Precisión = 0.3666666666666667
sci.electronics: 
	 [0.1 0.6 0.8] - Precisión = 0.5
sci.med: 
	 [1.  0.1 0.5] - Precisión = 0.5333333333333333
sci.space: 
	 [0.6 1.  0.6] - Precisión = 0.7333333333333334
soc.religion.christian: 
	 [1.  0.1 0.4] - Precisión = 

**¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?**

Si exite una diferencia en los valores, esto puede deberse a que no hay suficientes datos comparables en el conjunto de entrenamiento para poder clasficar correctamente todas las clases presentes.

**Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.**

Para exhaustividad 3 la peor clase consultada ha sido: `comp.sys.mac.hardware` con una precisión media del 0.0, pero con una exhaustividad de 10 se tiene como peor `talk.religion.misc` con una precision media de 0.16666666666666666.

A continuación mostramos los mensajes de cada clase en la que se ha fallado:

In [24]:
imprimirMensajes(consultas_a_generar,documentos_guardados,'comp.sys.mac.hardware',idsEx3, resultadoEx3, nMensajesEx3, comparador=min)

Clase Consultada: comp.sys.mac.hardware


Mensaje Consultado: 

From: jacobs@cerritos.edu
Subject: Problem printing Quark on a SWII
Organization: Cerritos College, Norwalk CA
Lines: 15

Iv'e got a problem printing with a StyleWriterII. I am printing from a IIvx
with 20 megs ram. I am trying to print a Quark file that has 2 fonts a couple
of boxes and 3 gradient fills. 

Two things happen: I get a " Disk is full" error, that I can't find documented,
I also have parts of letters that are over one of the gradient fills get cut
off. This only happens to the text over the fill. Text adjecent in a different
box is uneffected.

Any ideas?

Thanks for the help...-- 
_______________________________________________________________________________
Karl Jacobs / jacobs@cerritos.edu ---------------------------------------------
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



Clase Obtenida: comp.os.ms-windows.misc


Mensaje Obtenido: 

From: rcs1@crux3.cit.cornell

**¿Con qué clases se ha confundido más dicha consulta?**

La consulta `comp.sys.mac.hardware` se ha confundido con `comp.os.ms-windows.misc` y es entendible ya que ambos pueden tener textos relacionados.

**¿A qué crees que se deben los malos resultados?**

Esto sucede porque en el caso de exhaustividad 3 se comparar los 3 mensajes de cada clase con el top 3 mensajes obtenidos por el buscador, sin embargo con exhaustividad 10 se caparan los 3 mensajes con el top 10 dandonos así valores mas cercanos a la realidad de precisión media.

## Apartado B

In [25]:
# Creamos el CountVectorizer
tfidfer= TfidfTransformer()

# Calculamos el valor TF-IDF 
documentos_preprocessed=tfidfer.fit_transform(documentos)

In [26]:
# Calculamos el valor TF-IDF 
# Al usar "transform" toma como IDF el del conjunto de entrenamiento 
consultas_preprocessed=tfidfer.transform(consultas)

In [27]:
resultadoEx3TFIDF, idsEx3TFIDF, nMensajesEx3TFIDF = ejecutarBuscador(consultas_a_generar,consultas_preprocessed, documentos_guardados,documentos_preprocessed,3)

Resultados Precisión

alt.atheism: 
	 [0.66666667 0.         0.        ] - Precisión = 0.2222222222222222
comp.graphics: 
	 [0.33333333 0.         0.        ] - Precisión = 0.1111111111111111
comp.os.ms-windows.misc: 
	 [0.33333333 0.         0.        ] - Precisión = 0.1111111111111111
comp.sys.ibm.pc.hardware: 
	 [1.         0.66666667 0.33333333] - Precisión = 0.6666666666666666
comp.sys.mac.hardware: 
	 [0.         0.33333333 0.66666667] - Precisión = 0.3333333333333333
comp.windows.x: 
	 [0.         0.66666667 0.33333333] - Precisión = 0.3333333333333333
misc.forsale: 
	 [0.33333333 0.66666667 0.33333333] - Precisión = 0.4444444444444444
rec.autos: 
	 [0.         0.33333333 1.        ] - Precisión = 0.4444444444444444
rec.motorcycles: 
	 [1.         1.         0.33333333] - Precisión = 0.7777777777777778
rec.sport.baseball: 
	 [1. 1. 1.] - Precisión = 1.0
rec.sport.hockey: 
	 [1. 1. 1.] - Precisión = 1.0
sci.crypt: 
	 [1.         0.33333333 0.33333333] - Precisión = 0.555555555555

In [28]:
resultadoEx10TFIDF, idsEx10TFIDF, nMensajesEx10TFIDF = ejecutarBuscador(consultas_a_generar,consultas_preprocessed, documentos_guardados,documentos_preprocessed,10)

Resultados Precisión

alt.atheism: 
	 [0.7 0.1 0. ] - Precisión = 0.26666666666666666
comp.graphics: 
	 [0.6 0.3 0. ] - Precisión = 0.3
comp.os.ms-windows.misc: 
	 [0.7 0.1 0. ] - Precisión = 0.26666666666666666
comp.sys.ibm.pc.hardware: 
	 [1.  0.6 0.1] - Precisión = 0.5666666666666668
comp.sys.mac.hardware: 
	 [0.  0.2 0.4] - Precisión = 0.20000000000000004
comp.windows.x: 
	 [0.1 0.7 0.5] - Precisión = 0.4333333333333333
misc.forsale: 
	 [0.5 0.5 0.2] - Precisión = 0.39999999999999997
rec.autos: 
	 [0.2 0.4 0.9] - Precisión = 0.5
rec.motorcycles: 
	 [1.  0.9 0.3] - Precisión = 0.7333333333333333
rec.sport.baseball: 
	 [1.  0.6 1. ] - Precisión = 0.8666666666666667
rec.sport.hockey: 
	 [1. 1. 1.] - Precisión = 1.0
sci.crypt: 
	 [0.9 0.1 0.1] - Precisión = 0.3666666666666667
sci.electronics: 
	 [0.3 0.7 0.8] - Precisión = 0.6
sci.med: 
	 [1.  0.2 0.8] - Precisión = 0.6666666666666666
sci.space: 
	 [0.7 1.  0.9] - Precisión = 0.8666666666666667
soc.religion.christian: 
	 [1.  0.1 0.7] 

**¿Han cambiado los valores de precisión media para las clases del conjunto de datos?¿Qué clases han mejorado? ¿Cuáles han empeorado?**

Si han cambiado los valores, utilizando la función `comparar` podemos observar la diferencia entre los resultados de dos buscadores, en este caso observaremos la diferencia entre las precisiones medias donde si la columna mejora es positivo el resultado ha mejorado y viceversa.

### Comparación con Exhaustividad 3

In [29]:
comparar(resultadoEx3, resultadoEx3TFIDF)

,Clase,Mejora
0,alt.atheism,-0.111111
1,comp.graphics,-0.111111
2,comp.os.ms-windows.misc,-0.111111
3,comp.sys.ibm.pc.hardware,0.111111
4,comp.sys.mac.hardware,0.333333
5,comp.windows.x,0.000000
6,misc.forsale,-0.333333
7,rec.autos,0.111111
8,rec.motorcycles,-0.111111
9,rec.sport.baseball,0.222222


Como podemos observar en la tabla anterior, existen 5 clases que empeoran sus resultados, 6 clases que mantienen la misma precisión media y 9 clases que tienen una mejora en la precisión medio.

En general las busquedas con TF/IDF producen resultados positivos a pesar de que existen varias clases que se mantienen igual o incluso empeoran sus resultados.

In [30]:
comparar(resultadoEx10, resultadoEx10TFIDF)

,Clase,Mejora
0,alt.atheism,0.033333
1,comp.graphics,0.100000
2,comp.os.ms-windows.misc,-0.033333
3,comp.sys.ibm.pc.hardware,0.066667
4,comp.sys.mac.hardware,0.133333
5,comp.windows.x,0.133333
6,misc.forsale,-0.166667
7,rec.autos,0.200000
8,rec.motorcycles,0.000000
9,rec.sport.baseball,0.100000


Como podemos observar en la tabla anterior, existen 2 clases que empeoran sus resultados, 2 clases que mantienen la misma precisión media y 16 clases que tienen una mejora en la precisión medio.

En general las busquedas con TF/IDF producen una mejora en los resultados obtenidos.

**Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.**

In [31]:
imprimirMensajes(consultas_a_generar,documentos_guardados,'talk.politics.mideast',
                 idsEx3TFIDF, resultadoEx3TFIDF, nMensajesEx3TFIDF, comparador=max)


Clase Consultada: talk.politics.mideast


Mensaje Consultado: 

From: tclock@orion.oac.uci.edu (Tim Clock)
Subject: Re: Peace Talks
Nntp-Posting-Host: orion.oac.uci.edu
Organization: University of California, Irvine
Lines: 38

In article <1993May13.201441.23139@nysernet.org> astein@nysernet.org (Alan Stein) writes:
>It seems that, to keep the peace talks going, Israel has to keep
>making goodwill gesture after goodwill gesture, while Palestinian
>Arabs continue to go around hunting Jews.

You *know* that putting something like this out on the newsgroup is *only*
going to generate flames, not discussion. Try adding some substance to
the issue of "gestures" you mentioned.
>
>If the peace talks are going to have any realistic chance of success,
>the Arabs are going to have to start reciprocating, especially since
>they are the ones who will be getting tangible concessions in return
>for giving up only intangibles.  

What is it you feel that Israel *has* offered as a "gesture"? What would

In [32]:
imprimirMensajes(consultas_a_generar,documentos_guardados,'talk.politics.mideast',
                 idsEx3, resultadoEx3, nMensajesEx3, comparador=max)


Clase Consultada: talk.politics.mideast


Mensaje Consultado: 

From: bakken@cs.arizona.edu (Dave Bakken)
Subject: Re: Saudi clergy condemns debut of human rights group!
Keywords: international, non-usa government, government, civil rights, 	social issues, politics
Organization: U of Arizona CS Dept, Tucson
Lines: 101

In article <benali.737307554@alcor> benali@alcor.concordia.ca ( ILYESS B. BDIRA ) writes:
>It looks like Ben Baz's mind and heart are also blind, not only his eyes.
>I used to respect him, today I lost the minimal amount of respect that
>I struggled to keep for him.
>To All Muslim netters: This is the same guy who gave a "Fatwah" that
>Saudi Arabia can be used by the United Ststes to attack Iraq . 

They were attacking the Iraqis to drive them out of Kuwait,
a country whose citizens have close blood and business ties
to Saudi citizens.  And me thinks if the US had not helped out
the Iraqis would have swallowed Saudi Arabia, too (or at 
least the eastern oilfields).  And 

Como podemos observar, la clase `talk.politics.mideast` ha mejorado `0.555556` y si miramos los mensajes sin el TF/IDF lo confundía con mensajes de la clase `talk.politics.misc` y `sci.crypt` pero cuando aplicamos el TF/IDF 
recupera mayor cantidad de mensajes de la clase consultada. Esto se debe a que el TF/IDF hace que la importancia de una palabra sea inversamente relativa a lo común que es en el documentoy por tanto consigue una mejor tasa de aciertos.